In [1]:
import findspark
findspark.init()

In [33]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as T
from pyspark.sql import Row

In [3]:
import os

In [4]:
HOME = os.path.expanduser("~")
DATA_DIR = os.path.join(HOME,r'Data')

In [5]:
prices_dir = os.path.join(DATA_DIR,r'quant/prices')
prices_dir

'/Users/yevgeniy/Data/quant/prices'

In [6]:
spark=SparkSession.builder.master('spark://pop-os.localdomain:7077').appName('DataFrame-Kafka').getOrCreate()

In [7]:
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1635364539742'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '50000'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1635364540537'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.warehouse.dir',
  'file:/Users/yevgeniy/Development/projects/data-engineering/data-engineering/notebooks/spark-warehouse'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '192.168.0.25')]

In [8]:
sdf_prices = spark.read.csv(os.path.join(prices_dir,'prices_5yrs.csv'), header=True,inferSchema=True)

In [9]:
sdf_prices.printSchema()

root
 |-- date: string (nullable = true)
 |-- ticker: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)



In [45]:
sdf_prices.show(5)

+----------+------+------------------+------------------+------------------+------------------+---------+
|      date|ticker|              open|              high|               low|             close|   volume|
+----------+------+------------------+------------------+------------------+------------------+---------+
|2016-04-11|  AAPL|25.338489526693984|25.719833996666296| 25.30593586028477|25.350114822387695|117630000|
|2016-04-12|  AAPL|25.424524132308605|25.694256545777197| 25.26640733356858| 25.68030548095703|108929200|
|2016-04-13|  AAPL|25.764012071449244| 26.12210236496072|25.764012071449244|26.052345275878906|133029200|
|2016-04-14|  AAPL| 25.95468547387539|26.133730628820437|25.887252376182882| 26.06629753112793|101895600|
|2016-04-15|  AAPL| 26.06862375416007|26.112804488531896|25.515209673368787| 25.54311180114746|187756000|
+----------+------+------------------+------------------+------------------+------------------+---------+
only showing top 5 rows



In [15]:
sdf_tickers = spark.read.csv(os.path.join(prices_dir,'yahoo_tickers.csv'), header=True,inferSchema=True)

In [16]:
sdf_tickers.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Exchange: string (nullable = true)
 |-- Category Name: string (nullable = true)
 |-- Country: string (nullable = true)



In [19]:
sdf_tickers.show(5,False)

+------+---------------------------------------+--------+---------------------------+-------+
|Ticker|Name                                   |Exchange|Category Name              |Country|
+------+---------------------------------------+--------+---------------------------+-------+
|OEDV  |Osage Exploration and Development, Inc.|PNK     |null                       |USA    |
|AAPL  |Apple Inc.                             |NMS     |Electronic Equipment       |USA    |
|BAC   |Bank of America Corporation            |NYQ     |Money Center Banks         |USA    |
|AMZN  |Amazon.com, Inc.                       |NMS     |Catalog & Mail Order Houses|USA    |
|T     |AT&T Inc.                              |NYQ     |Telecom Services - Domestic|USA    |
+------+---------------------------------------+--------+---------------------------+-------+
only showing top 5 rows



In [35]:
sdf_tickers.withColumn("FullName",(T.concat(T.col("Ticker"),T.col("Name"))))\
    .select(T.col("FullName")).show(5,truncate=False)


+-------------------------------------------+
|FullName                                   |
+-------------------------------------------+
|OEDVOsage Exploration and Development, Inc.|
|AAPLApple Inc.                             |
|BACBank of America Corporation             |
|AMZNAmazon.com, Inc.                       |
|TAT&T Inc.                                 |
+-------------------------------------------+
only showing top 5 rows



### Joining

### Creating

    * From Row object

In [34]:
rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
authors_df = spark.createDataFrame(rows, ["Authors", "State"])
authors_df.show()

+-------------+-----+
|      Authors|State|
+-------------+-----+
|Matei Zaharia|   CA|
|  Reynold Xin|   CA|
+-------------+-----+



### Select

In [31]:
sdf_prices.sort(T.col('date').desc())

DataFrame[date: string, ticker: string, open: double, high: double, low: double, close: double, volume: int]

In [50]:
sdf_prices.select('ticker').show(1)

+------+
|ticker|
+------+
|  AAPL|
+------+
only showing top 1 row



In [49]:
sdf_prices.select(sdf_prices['ticker'] == 'AAPL').show(1)

+---------------+
|(ticker = AAPL)|
+---------------+
|           true|
+---------------+
only showing top 1 row



In [58]:
sdf_prices.filter('open > 25').select(['date','open']).show(1)

+----------+------------------+
|      date|              open|
+----------+------------------+
|2016-04-11|25.338489526693984|
+----------+------------------+
only showing top 1 row



#### Iteration
When you want to iterate through a DataFrame, you could use iterrows in pandas. In Spark, you create an array of rows using collect().

The following code will use a filter method to get all people over open > 25 and print the array:

In [65]:
top_25_prices = sdf_prices.filter('open > 25').select(['date','open']).collect()

In [67]:
top_25_prices[0]

Row(date='2016-04-11', open=25.338489526693984)

To get a single row, you can just pass the index. You can convert the row into different formats, and in this example, I converted it into a dictionary. As a dictionary, you can select any value by specifying the key. The code is shown as follows:

In [69]:
for r in top_25_prices:
    print(r.asDict())
    break

{'date': '2016-04-11', 'open': 25.338489526693984}


### Spark SQL

If you are more comfortable with SQL, you can filter a DataFrame using spark.sql. To use SQL, you must first create a view, then you can query it with SQL, as shown in the following code:

In [71]:
sdf_prices.createOrReplaceTempView('top_prices')
top_25_prices_view = spark.sql('select * from top_prices where open > 25')

In [72]:
top_25_prices_view.show(1)

+----------+------+------------------+------------------+-----------------+------------------+---------+
|      date|ticker|              open|              high|              low|             close|   volume|
+----------+------+------------------+------------------+-----------------+------------------+---------+
|2016-04-11|  AAPL|25.338489526693984|25.719833996666296|25.30593586028477|25.350114822387695|117630000|
+----------+------+------------------+------------------+-----------------+------------------+---------+
only showing top 1 row



#### Analysis

In [74]:
sdf_prices.describe('open').show()

+-------+------------------+
|summary|              open|
+-------+------------------+
|  count|             12590|
|   mean|409.02570375165925|
| stddev| 599.4995554732652|
|    min| 21.05486413111501|
|    max|            3547.0|
+-------+------------------+



In [77]:
sdf_prices.groupBy('ticker').count().show()

+------+-----+
|ticker|count|
+------+-----+
|  AAPL| 1259|
|   JPM| 1259|
|  TSLA| 1259|
|  GOOG| 1259|
|    FB| 1259|
|  NFLX| 1259|
|   WMT| 1259|
|  AMZN| 1259|
|  MSFT| 1259|
|  ADBE| 1259|
+------+-----+



In [81]:
# Aggregate by various operations with dict
sdf_prices.agg({'ticker' : 'count', 'open' : 'mean'}).show()

+-------------+------------------+
|count(ticker)|         avg(open)|
+-------------+------------------+
|        12590|409.02570375165925|
+-------------+------------------+



For both groupBy and agg, you can use mean, max, min, sum, and other methods
that you can read about in the documentation. There is a large number of other functions you can use that require you to import the pyspark.sql.functions module.
The following code imports it as f and demonstrates some useful functions. 